In [1]:
import pandas as pd
import os
import sys
import numpy as np
import tensorflow as tf
import tensorflow.keras as krs
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from tqdm.autonotebook import tqdm
from tqdm import trange

In [2]:
tf.__version__

'2.0.0'

In [3]:
%matplotlib inline

In [4]:
%load_ext autoreload

%autoreload 2

In [5]:
if os.getcwd().endswith("notebooks"):
    os.chdir(os.pardir)

In [6]:
sys.path.append(os.getcwd())

In [82]:
from src.data.train_data_utils import prepare_experiment_data
from src.models.training_utils import generator_ratings, generator_ratings_features, mae_masked, mape_masked, mse_masked
from src.models.train_model import get_collabfiltering_model1, build_shallow_autorec_single_input, build_autorec_multi_input2, get_array

# Data prep

In [8]:
exd = prepare_experiment_data()

Processed: 0.0%
Processed: 0.27373261797875836%
Processed: 0.5474652359575167%
Processed: 0.8211978539362751%
Processed: 0.0%
Processed: 0.4555393586005831%
Processed: 0.9110787172011662%
Processed: 0.0%
Processed: 0.6804109682248077%


In [10]:
ytest = np.expand_dims(exd.Xraw_test.rating.to_numpy(), axis=1)

# Model prep

## Collaborative filtering with bias

In [85]:
dim_embed = 30
nusers = max(exd.Xraw_train.user_id.max(), exd.Xraw_test.user_id.max(), exd.Xraw_valid.user_id.max())
nitems = exd.Xraw_train.item_id.max()

user_input = krs.Input(shape=(1,), name="user_input", dtype='int32')
u = krs.layers.Embedding(nusers + 1, dim_embed)(user_input)
u = krs.layers.Flatten()(u)
u_bias = tf.keras.layers.Flatten()(krs.layers.Embedding(nusers + 1, 1)(user_input))

item_input = krs.Input(shape=(1,), name='item_input', dtype='int32')
i = krs.layers.Embedding(nitems + 1, dim_embed)(item_input)
i = krs.layers.Flatten()(i)
i_bias = krs.layers.Flatten()(krs.layers.Embedding(nitems + 1, 1)(item_input))

ui = krs.layers.Dot(axes=(1,1))([u, i])

all_features = krs.layers.Concatenate()([ui, u_bias, i_bias])
drop1 = krs.layers.Dropout(0.4)(all_features)
out = krs.layers.Dense(1)(drop1)

model_mf = krs.Model(inputs=[user_input, item_input], outputs=out)
model_mf.compile(loss=mse_masked, optimizer='adam', metrics=[mse_masked, 'mae', mape_masked])

In [86]:
model_mf.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 1, 30)        610050      user_input[0][0]                 
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 1, 30)        196680      item_input[0][0]                 
____________________________________________________________________________________________

In [87]:
x_mf = [
        exd.Xraw_train.user_id.to_numpy(),
        exd.Xraw_train.item_id.to_numpy()
    ]
y_mf = exd.Xraw_train.rating.to_numpy()

x_val_mf = [
    exd.Xraw_valid.user_id.to_numpy(),
    exd.Xraw_valid.item_id.to_numpy()
]
y_val_mf = exd.Xraw_valid.rating.to_numpy()



In [88]:
modelcf_history = model_mf.fit(x=x_mf, y=y_mf, batch_size=32, epochs=10, validation_data=(x_val_mf, y_val_mf))

Train on 36532 samples, validate on 14697 samples
Epoch 1/10
36532/36532 [==============================] - 18s 481us/sample - loss: 14.3281 - mse_masked: 14.3262 - mae: 3.5742 - mape_masked: 0.8200 - val_loss: 9.8200 - val_mse_masked: 9.8146 - val_mae: 2.9354 - val_mape_masked: 0.6569
Epoch 2/10
36532/36532 [==============================] - 15s 400us/sample - loss: 6.2028 - mse_masked: 6.2021 - mae: 2.2402 - mape_masked: 0.5371 - val_loss: 4.4724 - val_mse_masked: 4.4696 - val_mae: 1.9376 - val_mape_masked: 0.4816
Epoch 3/10
36532/36532 [==============================] - 15s 404us/sample - loss: 2.6774 - mse_masked: 2.6770 - mae: 1.3924 - mape_masked: 0.3986 - val_loss: 2.9375 - val_mse_masked: 2.9362 - val_mae: 1.5736 - val_mape_masked: 0.4323
Epoch 4/10
36532/36532 [==============================] - 19s 508us/sample - loss: 1.6099 - mse_masked: 1.6097 - mae: 1.0404 - mape_masked: 0.3332 - val_loss: 2.4808 - val_mse_masked: 2.4802 - val_mae: 1.4509 - val_mape_masked: 0.4182
Epoch 5/

In [91]:
yhat_mf = model_mf.predict(x=[
    exd.Xraw_test.user_id.to_numpy(dtype='int32'),
    exd.Xraw_test.item_id.to_numpy(dtype='int32')
])

In [92]:
mape_masked(ytest, yhat_mf).numpy()

0.39190078

## Deep recommender

In [93]:
from sklearn.preprocessing import StandardScaler

ytrain_mu = np.mean(exd.Xraw_train.rating)
ytrain_norm = (exd.Xraw_train.rating - ytrain_mu).to_numpy()
yval_norm = (exd.Xraw_valid.rating - ytrain_mu).to_numpy()
ytest_norm = (exd.Xraw_test.rating - ytrain_mu).to_numpy()

In [94]:
exd.Xfeatures_train.shape

(36532, 57)

In [253]:
dim_embed = 30
nusers = max(exd.Xraw_train.user_id.max(), exd.Xraw_test.user_id.max(), exd.Xraw_valid.user_id.max())
nitems = exd.Xraw_train.item_id.max()

f_input = krs.Input(shape=(exd.Xfeatures_train.shape[1],), name="features_input", dtype='float32')
#f_drop = krs.layers.Dropout(rate=0.5)(f_input)
f_dense = krs.layers.Dense(4, activation='relu', kernel_initializer=krs.initializers.GlorotNormal())(f_input) #, kernel_regularizer=krs.regularizers.l1_l2(l1=0.001, l2=0.001))(f_input)


user_input = krs.Input(shape=(1,), name="user_input", dtype='int32')
u = krs.layers.Embedding(nusers + 1, dim_embed)(user_input)
u = krs.layers.Flatten()(u)
u_bias = tf.keras.layers.Flatten()(krs.layers.Embedding(nusers + 1, 1)(user_input))

item_input = krs.Input(shape=(1,), name='item_input', dtype='int32')
i = krs.layers.Embedding(nitems + 1, dim_embed)(item_input)
i = krs.layers.Flatten()(i)
i_bias = krs.layers.Flatten()(krs.layers.Embedding(nitems + 1, 1)(item_input))

ui = krs.layers.Dot(axes=(1,1))([u, i])
#ui = krs.layers.Multiply()([u, i])

all_features = krs.layers.Concatenate(axis=1)([ui, f_dense, u_bias, i_bias])
#drop =  krs.layers.Dropout(rate=0.2)(all_features)
d1 = krs.layers.Dense(8, activation="relu", kernel_initializer=krs.initializers.GlorotNormal())(all_features) #,  kernel_regularizer=krs.regularizers.l1_l2(l1=0.001, l2=0.001))(all_features)
#drop2 = krs.layers.Dropout(rate=0.2)(d1)
d2 = krs.layers.Dense(4, activation="relu", kernel_initializer=krs.initializers.GlorotNormal())(d1) #,  kernel_regularizer=krs.regularizers.l1_l2(l1=0.001, l2=0.001))(d1)
out = krs.layers.Dense(1, )(d2)

model = krs.Model(inputs=[user_input, item_input, f_input], outputs=out )
model.compile(loss=mse_masked, optimizer='adam', metrics=[mse_masked, 'mae', mape_masked])

In [254]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 30)        610050      user_input[0][0]                 
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1, 30)        196680      item_input[0][0]                 
______________________________________________________________________________________________

In [255]:
x = [
        exd.Xraw_train.user_id.to_numpy(),
        exd.Xraw_train.item_id.to_numpy(),
        exd.Xfeatures_train
    ]
y = exd.Xraw_train.rating.to_numpy()

x_val = [
    exd.Xraw_valid.user_id.to_numpy(),
    exd.Xraw_valid.item_id.to_numpy(),
    exd.Xfeatures_valid
]
y_val = exd.Xraw_valid.rating.to_numpy()

In [256]:
es = krs.callbacks.EarlyStopping(monitor='val_mape_masked', mode='min', min_delta=0.005, patience=3)
model_history = model.fit(x=x, y=y, batch_size=16, epochs=10, validation_data=(x_val, y_val), callbacks=[es])

Train on 36532 samples, validate on 14697 samples
Epoch 1/10
36532/36532 [==============================] - 33s 903us/sample - loss: 2.5064 - mse_masked: 2.5057 - mae: 1.1623 - mape_masked: 0.4318 - val_loss: 1.3356 - val_mse_masked: 1.3362 - val_mae: 0.9141 - val_mape_masked: 0.3737
Epoch 2/10
36532/36532 [==============================] - 30s 834us/sample - loss: 0.7715 - mse_masked: 0.7713 - mae: 0.6278 - mape_masked: 0.2740 - val_loss: 1.4685 - val_mse_masked: 1.4688 - val_mae: 1.0163 - val_mape_masked: 0.3765
Epoch 3/10
36532/36532 [==============================] - 31s 839us/sample - loss: 0.2080 - mse_masked: 0.2080 - mae: 0.2967 - mape_masked: 0.1228 - val_loss: 1.4265 - val_mse_masked: 1.4269 - val_mae: 0.9721 - val_mape_masked: 0.3725
Epoch 4/10
36532/36532 [==============================] - 31s 857us/sample - loss: 0.0720 - mse_masked: 0.0720 - mae: 0.1617 - mape_masked: 0.0594 - val_loss: 1.4637 - val_mse_masked: 1.4640 - val_mae: 1.0155 - val_mape_masked: 0.3752


In [260]:
yhat = model.predict(x=[
    exd.Xraw_test.user_id.to_numpy(dtype='int32'),
    exd.Xraw_test.item_id.to_numpy(dtype='int32'),
    exd.Xfeatures_test.astype('float32')
])
#yhat_denorm = yhat + ytrain_mu

In [261]:
mape_masked(ytest, yhat).numpy() #yhat_denorm).numpy()

0.38546914

In [259]:
import gc
tf.keras.backend.clear_session()
gc.collect()

29709